In [3]:
################################################### 모 듈 ###################################################

import pymysql as ps
from sqlalchemy import create_engine, text

def con():
    conn = ps.connect(host='localhost', user='root', passwd='root', db='auto_farming')
    return conn

def insert_db(table, calumns, values):
    conn = con()
    curs = conn.cursor()
    sql = f"insert into {table} {calumns} values {values};"
    curs.execute(sql)
    conn.commit()
    curs.close()
    conn.close()
    return sql

def select_db(calumn,table):
    conn = con()
    curs = conn.cursor()
    sql = f"select {calumn} from {table}"
    curs.execute(sql)
    result = curs.fetchall()
    print(result)
    curs.close()
    conn.close()
    return sql

class DAO():
    # 자동 수동 update 함수
    def auto_update(self):
        farm_id = request.args['farm_id'] # get방식으로 들어온 Key가 'farm_id'인 값을 변수에 담기
        am = request.args['am'] 
        conn = con()
        curs = conn.cursor()
        sql = f'update farm_table set auto = "{am}" where farm_id = "{farm_id}"'
        curs.execute(sql)
        conn.commit()
        auto_update_result = curs.fetchall()
        curs.close()
        conn.close()
        return ''

    # 자동인지 수동인지 db에서 받아오는 함수 
    def auto(self):
        farm_id = request.args['farm_id']
        conn = con()
        curs = conn.cursor()
        sql = f'select farm_id, auto from farm_table where farm_id = "{farm_id}"'
        curs.execute(sql)
        auto_result = curs.fetchall()
        curs.close()
        conn.close()
        auto_stt = auto_result[0][1]
#         print('auto_stt :',auto_result[0][1])
        return auto_stt


    # 자동 설정 값 받아오는 함수
    def auto_table(self):
        conn = con()
        curs = conn.cursor()
        sql = f'select max_tem, min_tem from auto_table'
        curs.execute(sql)
        auto_table_result = curs.fetchall()
        curs.close()
        conn.close()

        auto_max_tem = auto_table_result[0][0]
        auto_min_tem = auto_table_result[0][1]
#         print('auto_max_tem :',auto_table_result[0][0])
#         print('auto_min_tem :',auto_table_result[0][1])
        return auto_max_tem, auto_min_tem


    #수동 설정 값 수정해주는 함수
    def update(self):
        farm_id = request.args['farm_id']
        man_max_tem = request.args['max_tem']
        man_min_tem = request.args['min_tem']
        conn = con()
        curs = conn.cursor()
        sql = f'update manual_table set max_tem = {max_tem}, min_tem = {min_tem} where farm_id = "{farm_id}"'
        curs.execute(sql)
        conn.commit()
        result = curs.fetchall()
        curs.close()
        conn.close()
        return ''


    # 수동 설정 값 받아오는 함수
    def manual_table(self):
        conn = con()
        curs = conn.cursor()
        sql = f'select max_tem, min_tem from manual_table'
        curs.execute(sql)
        manual_table = curs.fetchall()
        curs.close()
        conn.close()
        man_max_tem = manual_table[0][0]
        man_min_tem = manual_table[0][1]
#         print('man_max_tem :',manual_table[0][0])
#         print('man_min_tem :',manual_table[0][1])
        return man_max_tem, man_min_tem


    # 뷰에서 농장 센서 값을 받아오는 함수
    def sensor(self):
        conn = con()
        curs = conn.cursor()
        sql = f'select * from sensor_view'
        curs.execute(sql)
        sensor_result = curs.fetchall()
        curs.close()
        conn.close()
        tem_stt = sensor_result[0][1]
        hum_stt = sensor_result[0][2]
#         print('tem_stt :',sensor_result[0][1])
#         print('hum_stt :',sensor_result[0][2])
        return tem_stt, hum_stt


    # 뷰에서 농장 엑추에이터 값을 받아오는 함수
    def actuator(self):
        conn = con()
        curs = conn.cursor()
        sql = f'select * from actuator_view'
        curs.execute(sql)
        actuator_result = curs.fetchall()
        curs.close()
        conn.close()
        ac_stt = actuator_result[0][1]
        vnt_stt = actuator_result[0][2]
#         print('ac_stt :',actuator_result[0][1])
#         print('vnt_stt :',actuator_result[0][2])
        return ac_stt, vnt_stt

In [ ]:
################################################### 원래 서버 ###################################################

from flask      import Flask, jsonify, request
from flask_cors import CORS, cross_origin
import pandas as pd
from datetime import datetime, timedelta
import time

app = Flask(__name__, static_folder='build')

@app.route('/')
def main():
    return "hi"
    
@app.route('/test', methods=['GET'])
def test():
    request.get_json()
    dao=DAO()
    dao.auto_update()
    dao.auto()
    dao.auto_table()
    dao.update()
    dao.manual_table()
    dao.sensor()
    dao.actuator()
    
    print('자동여부 :',auto_stt)
    print('자동최고온도설정 :',auto_max_tem)
    print('자동최저온도설정 :',auto_min_tem)
    print('수동최고온도설정 :',man_max_tem)
    print('수동최저온도설정 :',man_min_tem)    
    print('현재온도 :',tem_stt)
    print('현재습도 :',hum_stt)
    
    print('-'*14)
    if auto_stt == '1':   #자동으로 설정했을 때 제어하는 조건
        if int(auto_max_tem) < int(tem_stt):
            print("|에어컨 작동!|")
        if int(auto_max_tem)-5 > int(tem_stt):
            print("|에어컨 멈춰!|")   # 에어컨 한테 0번 시그널 보냄
    elif auto_stt == '0':   #수동으로 설정했을 때 제어하는 조건
        if int(man_max_tem) < int(tem_stt):
            print("|에어컨 작동!|")
        if int(man_max_tem)-5 > int(tem_stt):
            print("|에어컨 멈춰!|")
    print('-'*14)   
 
    return f'helow world'

In [ ]:
############################### 서버인데 함수리턴이 안되서 하드코딩으로 풀어둔 서버 ###############################

from flask      import Flask, jsonify, request
from flask_cors import CORS, cross_origin
import pandas as pd
from datetime import datetime, timedelta
import time

app = Flask(__name__, static_folder='build')

@app.route('/')
def main():
    return "hi"
    
@app.route('/test', methods=['GET'])
def test():
# get방식으로 들어온 코드 읽는 메소드 
    request.get_json()
    
# 자동 수동 update 함수
    am = request.args['am']
    farm_id = 'farm_id 01'
    conn = con()
    curs = conn.cursor()
    sql = f'update farm_table set auto = "{am}" where farm_id = "{farm_id}"'
    curs.execute(sql)
    conn.commit()
    auto_update_result = curs.fetchall()
    curs.close()
    conn.close()
    
# 자동인지 수동인지 db에서 받아오는 함수
    farm_id = 'farm_id 01'
    conn = con()
    curs = conn.cursor()
    sql = f'select farm_id, auto from farm_table where farm_id = "{farm_id}"'
    curs.execute(sql)
    auto_result = curs.fetchall()
    curs.close()
    conn.close()
    auto_stt = auto_result[0][1]
#     print('auto_stt :',auto_result[0][1])



# 자동 설정 값 받아오는 함수
    conn = con()
    curs = conn.cursor()
    sql = f'select max_tem, min_tem from auto_table'
    curs.execute(sql)
    auto_table_result = curs.fetchall()
    curs.close()
    conn.close()

    auto_max_tem = auto_table_result[0][0]
    auto_min_tem = auto_table_result[0][1]
#     print('-'*10)
#     print('auto_max_tem :',auto_table_result[0][0])
#     print('auto_min_tem :',auto_table_result[0][1])


#수동 설정 값 수정해주는 함수
    man_max_tem = request.args['max_tem']
    man_min_tem = request.args['min_tem']
    farm_id = 'farm_id 01'
    conn = con()
    curs = conn.cursor()
    sql = f'update manual_table set max_tem = {max_tem}, min_tem = {min_tem} where farm_id = "{farm_id}"'
    curs.execute(sql)
    conn.commit()
    result = curs.fetchall()
    curs.close()
    conn.close()


# 수동 설정 값 받아오는 함수
    conn = con()
    curs = conn.cursor()
    sql = f'select max_tem, min_tem from manual_table'
    curs.execute(sql)
    manual_table = curs.fetchall()
    curs.close()
    conn.close()
    man_max_tem = manual_table[0][0]
    man_min_tem = manual_table[0][1]
#     print('man_max_tem :',manual_table[0][0])
#     print('man_min_tem :',manual_table[0][1])
    

# 뷰에서 농장 센서 값을 받아오는 함수
    conn = con()
    curs = conn.cursor()
    sql = f'select * from sensor_view'
    curs.execute(sql)
    sensor_result = curs.fetchall()
    curs.close()
    conn.close()
    tem_stt = sensor_result[0][1]
    tem_stt = request.args['tem_stt']
    hum_stt = sensor_result[0][2]
#     print('tem_stt :',sensor_result[0][1])
#     print('hum_stt :',sensor_result[0][2])

# 뷰에서 농장 엑추에이터 값을 받아오는 함수
    conn = con()
    curs = conn.cursor()
    sql = f'select * from actuator_view'
    curs.execute(sql)
    actuator_result = curs.fetchall()
    curs.close()
    conn.close()
    ac_stt = actuator_result[0][1]
    vnt_stt = actuator_result[0][2]
#     print('ac_stt :',actuator_result[0][1])
#     print('vnt_stt :',actuator_result[0][2])
    
    print('자동여부 :',auto_stt)
    print('자동최고온도설정 :',auto_max_tem)
    print('자동최저온도설정 :',auto_min_tem)
    print('수동최고온도설정 :',man_max_tem)
    print('수동최저온도설정 :',man_min_tem)    
    print('현재온도 :',tem_stt)
    print('현재습도 :',hum_stt)
    
    print('-'*14)
    if auto_stt == '1':   #자동으로 설정했을 때 제어하는 조건
        if int(auto_max_tem) < int(tem_stt):
            print("|에어컨 작동!|")
        if int(auto_max_tem)-5 > int(tem_stt):
            print("|에어컨 멈춰!|")   # 에어컨 한테 0번 시그널 보냄
    elif auto_stt == '0':   #수동으로 설정했을 때 제어하는 조건
        if int(man_max_tem) < int(tem_stt):
            print("|에어컨 작동!|")
        if int(man_max_tem)-5 > int(tem_stt):
            print("|에어컨 멈춰!|")
    print('-'*14)   
 
    return f'hello world'

if __name__ == '__main__':
    app.run(host='localhost', port=9500)

In [ ]:
!pip install flask_cors

In [6]:
################################################# 시나리오 서버 #################################################

from flask      import Flask, jsonify, request
from flask_cors import CORS, cross_origin
import pandas as pd
from datetime import datetime, timedelta
import time

app = Flask(__name__, static_folder='build')

@app.route('/')
def main():
    return "hi"
    
@app.route('/test', methods=["GET","POST"])
def test():
#     request.get_json()
#     request.args['x1']
# 자동 수동 update 함수
    am = 1
    farm_id = 'farm_id 01'
    conn = con()
    curs = conn.cursor()
    sql = f'update farm_table set auto = "{am}" where farm_id = "{farm_id}"'
    curs.execute(sql)
    conn.commit()
    auto_update_result = curs.fetchall()
    curs.close()
    conn.close()
    
# 자동인지 수동인지 db에서 받아오는 함수
    farm_id = 'farm_id 01'
    conn = con()
    curs = conn.cursor()
    sql = f'select farm_id, auto from farm_table where farm_id = "{farm_id}"'
    curs.execute(sql)
    auto_result = curs.fetchall()
    curs.close()
    conn.close()
    auto_stt = auto_result[0][1]
#     print('auto_stt :',auto_result[0][1])



# 자동 설정 값 받아오는 함수
    conn = con()
    curs = conn.cursor()
    sql = f'select max_tem, min_tem from auto_table'
    curs.execute(sql)
    auto_table_result = curs.fetchall()
    curs.close()
    conn.close()
    auto_max_tem = auto_table_result[0][0]
    auto_min_tem = auto_table_result[0][1]
#     print('-'*10)
#     print('auto_max_tem :',auto_table_result[0][0])
#     print('auto_min_tem :',auto_table_result[0][1])


#수동 설정 값 수정해주는 함수
    max_tem = 100 #나중에 json으로 받아서 바꾸면됨 
    min_tem = 80 #나중에 json으로 받아서 바꾸면됨
    farm_id = 'farm_id 01'
    conn = con()
    curs = conn.cursor()
    sql = f'update manual_table set max_tem = {max_tem}, min_tem = {min_tem} where farm_id = "{farm_id}"'
    curs.execute(sql)
    conn.commit()
    result = curs.fetchall()
    curs.close()
    conn.close()


# 수동 설정 값 받아오는 함수
    conn = con()
    curs = conn.cursor()
    sql = f'select max_tem, min_tem from manual_table'
    curs.execute(sql)
    manual_table = curs.fetchall()
    curs.close()
    conn.close()
    man_max_tem = manual_table[0][0]
    man_min_tem = manual_table[0][1]
#     print('-'*10)
#     print('man_max_tem :',manual_table[0][0])
#     print('man_min_tem :',manual_table[0][1])
    

# 뷰에서 농장 센서 값을 받아오는 함수
    conn = con()
    curs = conn.cursor()
    sql = f'select * from sensor_view'
    curs.execute(sql)
    sensor_result = curs.fetchall()
    curs.close()
    conn.close()
    tem_stt = sensor_result[0][1]
    hum_stt = sensor_result[0][2]
#     print('-'*10)
#     print('tem_stt :',sensor_result[0][1])
#     print('hum_stt :',sensor_result[0][2])

# 뷰에서 농장 엑추에이터 값을 받아오는 함수
    conn = con()
    curs = conn.cursor()
    sql = f'select * from actuator_view'
    curs.execute(sql)
    actuator_result = curs.fetchall()
    curs.close()
    conn.close()
    ac_stt = actuator_result[0][1]
    vnt_stt = actuator_result[0][2]
#     print('-'*10)
#     print('ac_stt :',actuator_result[0][1])
#     print('vnt_stt :',actuator_result[0][2])
    
    print('자동여부 :',auto_stt)
    print('자동최고온도설정 :',auto_max_tem)
    print('자동최저온도설정 :',auto_min_tem)
    print('수동최고온도설정 :',man_max_tem)
    print('수동최저온도설정 :',man_min_tem)    
    print('현재온도 :',tem_stt)
    print('현재습도 :',hum_stt)
    
    print('-'*14)
    if auto_stt == '1':   #자동으로 설정했을 때 제어하는 조건
        if int(auto_max_tem) < int(tem_stt):
            print("|에어컨 작동!|")
        if int(auto_max_tem)-5 > int(tem_stt):
            print("|에어컨 멈춰!|")   # 에어컨 한테 0번 시그널 보냄
    elif auto_stt == '0':   #수동으로 설정했을 때 제어하는 조건
        if int(man_max_tem) < int(tem_stt):
            print("|에어컨 작동!|")
        if int(man_max_tem)-5 > int(tem_stt):
            print("|에어컨 멈춰!|")
    print('-'*14)   
    
    print('='*42)
    print("며칠 후 사용자는 수동모드를 쓰고 싶어졌다.")
    print("(수동모드로 설정)")
    auto_stt = '0' #이건 POST 방식 JSON으로 받아오는 것 
    print("수동최고온도 25도 설정")
    man_max_tem = 25
    print("수동최저온도 5도 설정")
    man_min_tem = 5
    print('='*42)
    
    print('자동여부 :',auto_stt)
    print('자동최고온도설정 :',auto_max_tem)
    print('자동최저온도설정 :',auto_min_tem)
    print('수동최고온도설정 :',man_max_tem)
    print('수동최저온도설정 :',man_min_tem)  
    tem_stt = 27 #들어왔다 치고 
    print('현재온도 :',tem_stt)
    print('현재습도 :',hum_stt)
   
    print('-'*14)
    if auto_stt == '1':   #자동으로 설정했을 때 제어하는 조건
        if int(auto_max_tem) < int(tem_stt):
            print("|에어컨 작동!|")
        if int(auto_max_tem)-5 > int(tem_stt):
            print("에어컨 멈춰!")   # 에어컨 한테 0번 시그널 보냄
    elif auto_stt == '0':   #수동으로 설정했을 때 제어하는 조건
        if int(man_max_tem) < int(tem_stt):
            print("|에어컨 작동!|")
        if int(man_max_tem)-5 > int(tem_stt):
            print("에어컨 멈춰!")
    print('-'*14)
    return f'hi'

if __name__ == '__main__':
    app.run(host='localhost', port=9501)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9501/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Oct/2022 10:41:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2022 10:41:46] "GET /test HTTP/1.1" 200 -


자동여부 : 1
자동최고온도설정 : 30
자동최저온도설정 : 10
수동최고온도설정 : 100
수동최저온도설정 : 80
현재온도 : 35.00
현재습도 : 3.00
--------------
|에어컨 작동!|
--------------
며칠 후 사용자는 수동모드를 쓰고 싶어졌다.
(수동모드로 설정)
수동최고온도 25도 설정
수동최저온도 5도 설정
자동여부 : 0
자동최고온도설정 : 30
자동최저온도설정 : 10
수동최고온도설정 : 25
수동최저온도설정 : 5
현재온도 : 27
현재습도 : 3.00
--------------
|에어컨 작동!|
--------------
